# BERTを用いた固有表現認識

このノートブックでは、BERTを使って固有表現認識をします。データセットとしてはCoNLL 2003を使います。なお、学習にはGPUを積んだマシンを使うことを推奨します。Colabであれば、「ランタイム」から「ランタイムのタイプを変更」を選択し、GPUに変更してください。

## 準備

### パッケージのインストール

In [1]:
!pip install -q transformers==4.10.2 datasets==1.12.1 seqeval==1.2.2

     |████████████████████████████████| 2.8 MB 5.3 MB/s 
     |████████████████████████████████| 270 kB 35.4 MB/s 
     |████████████████████████████████| 43 kB 1.4 MB/s 
     |████████████████████████████████| 3.3 MB 3.4 MB/s 
     |████████████████████████████████| 895 kB 43.1 MB/s 
     |████████████████████████████████| 636 kB 41.1 MB/s 
     |████████████████████████████████| 52 kB 1.5 MB/s 
     |████████████████████████████████| 119 kB 42.4 MB/s 
     |████████████████████████████████| 243 kB 48.2 MB/s 
     |████████████████████████████████| 1.3 MB 39.8 MB/s 
     |████████████████████████████████| 294 kB 46.4 MB/s 
     |████████████████████████████████| 142 kB 43.5 MB/s 


### インポート

In [2]:
import numpy as np

from datasets import load_dataset, load_metric
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification

### データセットの読み込み

データセットを用意するために、Huggingfaceの[Datasets](https://github.com/huggingface/datasets)ライブラリを使いましょう。このライブラリを使うことで、データのダウンロードや読み込みをすることができます。たとえば、CoNLL 2003であれば、以下のように`load_dataset`関数に`conll2003`と渡すことで、ダウンロードと読み込みが完了します。

In [3]:
datasets = load_dataset("conll2003")

Downloading:   0%|          | 0.00/2.60k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/650k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/163k [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/40e7cb6bcc374f7c349c83acd1e9352a4f09474eb691f64f364ee62eb65d0ca6. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

読み込んだオブジェクトは[`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict)になっています。`DatasetDict`を使うと、学習・検証・テスト用のデータセットにアクセスできます。

In [4]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

実際のデータにアクセスするには、分割名（train, validation, test）のあとに、インデックスを指定します。

In [5]:
datasets["train"][0]

{'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'id': '0',
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.']}

ラベルは既にID化されているため、モデルから簡単に利用できます。実際のラベルは、`features`へアクセスするとわかります。

In [6]:
datasets["train"].features["ner_tags"]

Sequence(feature=ClassLabel(num_classes=9, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], names_file=None, id=None), length=-1, id=None)

ラベルは`ClassLabel`のリストになっているので、実際のラベル名を得るためには、`feature`属性へアクセスします。

In [7]:
label_list = datasets["train"].features["ner_tags"].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

## 前処理

テキストをモデルに与える前に、前処理をします。入力をトークン化（トークンを事前学習済みモデルの語彙の対応するIDに変換することを含む）し、モデルが期待するフォーマットにするとともに、モデルが必要とするその他の入力を生成します。

これらの作業を行うために、`AutoTokenizer.from_pretrained`メソッドでトークナイザーをインスタンス化し、以下のことを確認します。

- 使用したいモデルのアーキテクチャに対応するトークナイザを取得
- 事前学習時に使用した語彙をダウンロード

In [8]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

トークナイザーにテキストを与えて、出力を確認してみましょう。

In [9]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [101, 7592, 1010, 2023, 2003, 2028, 6251, 999, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

もし入力が単語に分割済みの場合、トークナイザーの引数に`is_split_into_words=True`を指定して、単語のリストを渡します。

In [10]:
tokenizer(["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."], is_split_into_words=True)

{'input_ids': [101, 7592, 1010, 2023, 2003, 2028, 6251, 3975, 2046, 2616, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

事前学習済みのモデルを使う場合、サブワードレベルのトークナイザーが使われていることがあります。これはつまり、単語に分割されていたとしても、さらに分割される可能性があることを意味します。その例を見てみましょう。

In [11]:
example = datasets["train"][4]
print(example["tokens"])

['Germany', "'s", 'representative', 'to', 'the', 'European', 'Union', "'s", 'veterinary', 'committee', 'Werner', 'Zwingmann', 'said', 'on', 'Wednesday', 'consumers', 'should', 'buy', 'sheepmeat', 'from', 'countries', 'other', 'than', 'Britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.']


In [12]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'germany', "'", 's', 'representative', 'to', 'the', 'european', 'union', "'", 's', 'veterinary', 'committee', 'werner', 'z', '##wing', '##mann', 'said', 'on', 'wednesday', 'consumers', 'should', 'buy', 'sheep', '##me', '##at', 'from', 'countries', 'other', 'than', 'britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.', '[SEP]']


分割結果を見ると、「Zwingmann」と「sheepmeat」という単語が3つのサブワードに分割されていることがわかります。

分類タスクであればあまり困らないのですが、固有表現認識の場合、少々困ったことになります。トークナイザーが返す入力IDの系列が、データセットに含まれるラベルのリストよりも長くなるため、ラベルに対して何らかの処理をしなければならないのです。長くなるのは以下の理由によるものです。

- 特殊なトークンが追加される（上記では「[CLS]」と「[SEP]」が追加されている）
- 単語が複数のトークンに分割される可能性がある

結果として、以下に示すように、入力と出力の系列長が一致しなくなってしまいます。

In [13]:
len(example["ner_tags"]), len(tokenized_input["input_ids"])

(31, 39)

ありがたいことに、トークナイザーは `word_ids` メソッドを持っており、この出力が問題解決の役に立ちます。

In [14]:
print(tokenized_input.word_ids())

[None, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 10, 11, 11, 11, 12, 13, 14, 15, 16, 17, 18, 18, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, None]


見ての通り、サブワードに分割された入力IDと同じ数の要素を持つリストを返します。ここで、特殊なトークンを `None` に、その他のすべてのトークンをそれぞれの単語にマッピングします。つまり、サブワードと元の単語を対応付けられるのです。この出力結果をりようすると、ラベルを入力IDに合わせることができます。

In [15]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example["ner_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

39 39


ここでは、特殊なトークンのラベルを-100（PyTorchが無視するインデックス）に設定し、他のすべてのトークンのラベルを、元となった単語のラベルに設定しています。別の戦略として、ある単語から得られた最初のトークンにのみラベルを設定し、同じ単語から得られた他のサブトークンには-100のラベルを与えるというやり方もあります。ここでは、以下のフラグの値を変更するだけで、戦略を切り替えられるようにします。

In [16]:
label_all_tokens = True

これで、前処理の関数を書く準備ができました。入力データは、`truncation=True`（テキストを切り詰める）、`is_split_into_words=True`という引数を付けて、`tokenizer`に渡します。そして、選んだ戦略を用いて、ラベルをトークンIDに合わせます。

In [17]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # 特殊なトークンの場合-100を設定
            if word_idx is None:
                label_ids.append(-100)
            # 各単語の最初のトークンにはラベルを設定
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # 残りのトークンは戦略によって、-100か最初のトークンと同じラベルを設定するか切り替える
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

定義した関数には、複数のデータを与えられます。複数のデータを与えた場合、トークナイザーは各キーに対応する入れ子になったリストを返します。

In [18]:
tokenize_and_align_labels(datasets['train'][:5])

{'input_ids': [[101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012, 102], [101, 2848, 13934, 102], [101, 9371, 2727, 1011, 5511, 1011, 2570, 102], [101, 1996, 2647, 3222, 2056, 2006, 9432, 2009, 18335, 2007, 2446, 6040, 2000, 10390, 2000, 18454, 2078, 2329, 12559, 2127, 6529, 5646, 3251, 5506, 11190, 4295, 2064, 2022, 11860, 2000, 8351, 1012, 102], [101, 2762, 1005, 1055, 4387, 2000, 1996, 2647, 2586, 1005, 1055, 15651, 2837, 14121, 1062, 9328, 5804, 2056, 2006, 9317, 10390, 2323, 4965, 8351, 4168, 4017, 2013, 3032, 2060, 2084, 3725, 2127, 1996, 4045, 6040, 2001, 24509, 1012, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1, 1, 1

この関数をデータセット内のすべての文に適用するには、先ほど作成した `dataset` オブジェクトの `map` メソッドを使用します。これにより、`dataset`内のすべての分割（train, validation, test）のすべての要素にこの関数が適用されるので、学習・検証・テストデータの前処理が1つのコマンドで完了することになります。

In [19]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

なお、`batched=True`を渡すことで、テキストを一括してエンコードすることができます。こうすると、マルチスレッドを使ってバッチ内のテキストを同時に処理します。

## モデルの学習

データの準備ができたので、事前学習済みモデルをダウンロードして、Fine-tuneします。今回のタスクはトークンの分類に関するものなので、`AutoModelForTokenClassification`クラスを使用します。トークナイザーと同様に、`from_pretrained`メソッドがモデルをダウンロードしてキャッシュしてくれます。モデル以外にはラベル数を指定する必要があります。

In [20]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

モデルの学習方法にもいくつかありますが、よく使われているのは`Trainer`です。`Trainer`をインスタンス化するためには、いくつか用意しなければならないものがありますが。最も重要なのは[`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments) です。`TrainingArguments`は、学習をカスタマイズするためのクラスです。学習率やエポック数などはこのクラスで指定します。必ず指定しなければならないのは、モデルのチェックポイントを保存するためのフォルダ名で、その他の引数はすべてオプションです。

In [21]:
batch_size = 16
args = TrainingArguments(
    "ner-conll2003",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

ここでは、評価を各エポックの最後に行うように設定（`evaluation_strategy="epoch"`）し、学習率をデフォルト値から変更し、学習のエポック数と重みの減衰をカスタマイズしています。

次に、データコレーター（DataCollator）を定義します。データコレーターを使うことで、すべてのデータが同じ長さになるようにパディングを適用します。各パディングは、最も長いデータに合わせて行われます。入力だけでなくラベルもパディングされます。

In [22]:
data_collator = DataCollatorForTokenClassification(tokenizer)

トレーナー」で最後に定義しなければならないのは，予測値から指標を計算する方法です。ここでは、Datasetsライブラリを介して、[`seqeval`](https://github.com/chakki-works/seqeval)を読み込みます。`seqeval`は系列ラベリングの評価をするためによく使われているライブラリです。

In [23]:
metric = load_metric("seqeval")

Downloading:   0%|          | 0.00/2.48k [00:00<?, ?B/s]

この指標は、予測と正解ラベルのリストを受け取ります。そのため、予測値に少しだけ後処理をする必要があります。

- 各トークンの予測インデックス（最大ロジット）を選択する
- それを文字列ラベルに変換する
- ラベルに-100を設定してた場合、無視する

以下の関数は、メトリックを適用する前に、`Trainer.evaluate`の結果に対して、これらの後処理をします。

In [24]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

あとは、これらの情報をデータセットと一緒に`Trainer`に渡すだけです。

In [25]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

これで、`train`メソッドを呼び出すだけで、モデルをFine-tuneできます。

In [26]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, ner_tags, id, pos_tags, chunk_tags.
***** Running training *****
  Num examples = 14041
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2634


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.214800,0.061692,0.921064,0.933326,0.927155,0.983558
2,0.047000,0.058652,0.928336,0.943394,0.935804,0.985035
3,0.026300,0.057437,0.934327,0.946974,0.940608,0.985845


Saving model checkpoint to ner-conll2003/checkpoint-500
Configuration saved in ner-conll2003/checkpoint-500/config.json
Model weights saved in ner-conll2003/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ner-conll2003/checkpoint-500/tokenizer_config.json
Special tokens file saved in ner-conll2003/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, ner_tags, id, pos_tags, chunk_tags.
***** Running Evaluation *****
  Num examples = 3250
  Batch size = 16
Saving model checkpoint to ner-conll2003/checkpoint-1000
Configuration saved in ner-conll2003/checkpoint-1000/config.json
Model weights saved in ner-conll2003/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ner-conll2003/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ner-conll2003/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to n

TrainOutput(global_step=2634, training_loss=0.0768560467113938, metrics={'train_runtime': 932.1922, 'train_samples_per_second': 45.187, 'train_steps_per_second': 2.826, 'total_flos': 1019752160161410.0, 'train_loss': 0.0768560467113938, 'epoch': 3.0})

`evaluate`メソッドでは、評価用データセットや別のデータセットで再度評価を行うことができます。

In [27]:
trainer.evaluate(tokenized_datasets["test"])

The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, ner_tags, id, pos_tags, chunk_tags.
***** Running Evaluation *****
  Num examples = 3453
  Batch size = 16


{'epoch': 3.0,
 'eval_accuracy': 0.976745390511705,
 'eval_f1': 0.9007543611504008,
 'eval_loss': 0.12319803982973099,
 'eval_precision': 0.8946382580191993,
 'eval_recall': 0.9069546641348208,
 'eval_runtime': 19.9272,
 'eval_samples_per_second': 173.281,
 'eval_steps_per_second': 10.839}

各固有表現タイプの適合率/再現率/f1を得るには，`predict`メソッドの結果に先ほどと同じ関数を適用します．

In [28]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, ner_tags, id, pos_tags, chunk_tags.
***** Running Prediction *****
  Num examples = 3453
  Batch size = 16


{'LOC': {'f1': 0.9104271933853927,
  'number': 2124,
  'precision': 0.8887892376681614,
  'recall': 0.9331450094161958},
 'MISC': {'f1': 0.7670396744659206,
  'number': 996,
  'precision': 0.777319587628866,
  'recall': 0.7570281124497992},
 'ORG': {'f1': 0.8780581039755352,
  'number': 2588,
  'precision': 0.8687594553706506,
  'recall': 0.8875579598145286},
 'PER': {'f1': 0.9634416543574594,
  'number': 2718,
  'precision': 0.9670126019273536,
  'recall': 0.9598969830757911},
 'overall_accuracy': 0.976745390511705,
 'overall_f1': 0.9007543611504008,
 'overall_precision': 0.8946382580191993,
 'overall_recall': 0.9069546641348208}

## Reference

- [Token Classification](https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/token_classification.ipynb#scrollTo=FBiW8UpKIrJW)
- [Fine-tuning with custom datasets](https://huggingface.co/transformers/custom_datasets.html#tok-ner)